
<h1 align=center>Réseaux de neurones avec Keras</h1>

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras
import time

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
X_train.shape

In [ ]:
X_train[0]

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.show()

In [ ]:
y_train

In [ ]:
class_names = ["T-shirt", "Pantalon", "Pull", "Robe", "Manteau",
               "Sandale", "Chemise", "Sneaker", "Sac", "Bottines"]

In [ ]:
class_names[y_train[0]]

In [ ]:
lignes = 5
cols = 10
#plt.figure(figsize=(cols*1.4, lignes * 1.8))

for l in range(lignes):
    for c in range(cols):
        index = cols * l + c
        plt.subplot(lignes, cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        #plt.title(class_names[y_train[index]]+"\n"+str(y_train[index]))
plt.show()

In [ ]:
model = keras.models.Sequential()

In [ ]:
model.add(keras.layers.Flatten(input_shape=[28, 28]))

In [ ]:
model.add(keras.layers.Dense(300, activation="relu"))

In [ ]:
model.add(keras.layers.Dense(100, activation="relu"))

In [ ]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
#Autre manière d'instancier un reseau keras
"""
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
"""

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
hist= model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [ ]:
hist.history

In [ ]:
pd.DataFrame(hist.history).plot(figsize=(8,5))

In [ ]:
hist= model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [ ]:
test_loss, test_accuracy= model.evaluate(X_test, y_test)
print(f'Test accuracy {test_accuracy}')
print(f'Test_loss {test_loss}')

In [ ]:
X=X_test[:10]
lignes = 1
cols = 10
#plt.figure(figsize=(cols*1.4, lignes * 1.8))

for l in range(lignes):
    for c in range(cols):
        index = cols * l + c
        plt.subplot(lignes, cols, index + 1)
        plt.imshow(X[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        #plt.title(class_names[y_train[index]]+"\n"+str(y_train[index]))
plt.show()

In [ ]:
y_prob = model.predict(X)
y_prob.round(3)

In [ ]:
y_pred = y_prob.argmax(axis=1)
y_pred

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
X_valid_scaled = scaler.transform(X_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
X_test_scaled = scaler.transform(X_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [ ]:
model_sc = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_sc.compile(optimizer = 'sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')
hist2 = model_sc.fit(X_train_scaled, y_train, epochs=15,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
test_loss, test_accuracy= model_sc.evaluate(X_test_scaled, y_test)
print(f'Test accuracy {test_accuracy}')
print(f'Test_loss {test_loss}')

In [ ]:
pd.DataFrame(hist2.history).plot(figsize=(8,5))

In [ ]:
y_prob = model_sc.predict(X)
y_prob.round(3)

In [ ]:
y_pred = y_prob.argmax(axis=1)
y_pred

In [ ]:
n_rows = 1
n_cols = 10
plt.figure(figsize=(n_cols*1.4, n_rows * 1.8))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_test[index]]+"\n"+class_names[y_pred[index]])
plt.show()

In [ ]:
model_final = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_final.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd", metrics=["accuracy"])

In [ ]:
callbacks = [
    
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint("modele_mnist_fashion.h5", save_best_only=True),
]

In [ ]:
hist3 = model_final.fit(X_train_scaled, y_train, epochs=50,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=callbacks)

In [ ]:
model_reload = keras.models.load_model("modele_mnist_fashion.h5")

In [ ]:
model_reload.summary()

In [ ]:
y_prob = model_reload.predict(X)
y_prob.round(3)

In [ ]:
n_rows = 1
n_cols = 10
plt.figure(figsize=(n_cols*1.4, n_rows * 1.8))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_test[index]]+"\n"+class_names[y_pred[index]])
plt.show()

In [ ]:
# Chargement de l'extension Tensorboard pour notebook
%load_ext tensorboard

In [ ]:
#Definition du dossier des logs d'événements
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
#Fonction de lancement du serveur tensorboard
def tb(logdir=log_dir, port=6006, open_tab=True, sleep=2):
    import subprocess
    proc = subprocess.Popen(
        "tensorboard --logdir={0} --port={1}".format(logdir, port))
    if open_tab:
        import time
        time.sleep(sleep)
        import webbrowser
        webbrowser.open("http://127.0.0.1:{}/".format(port))
    return proc

In [ ]:
#Redéfinition du modèle et des callbacks
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd", metrics=["accuracy"])


callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint("my_mnist_model.h5", save_best_only=True),
]
history = model.fit(X_train_scaled, y_train, epochs=10,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=callbacks)

In [ ]:
#Lancement du serveur tensorboard
server =tb()

In [ ]:
#Arrêt du serveur
server.kill()